# Spark: understanding its storage system

In this tutorial, we will have a short introduction to Spark.

Let's start by loading the basic libraries and starting a **Spark session**. We install the package `findspark`, if you still don't have it.

In [ ]:
!python -m pip install findspark

In [ ]:
import findspark
import pyspark
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.getOrCreate()

# DataFrames: an introduction

In Spark, the underlying storage system relies on Resilient Distributed Datasets (RDDs), but we usually work with DataFrames, a higher-level API.

Let's create just a simple DataFrame with a single column (called 'number') and 1000 rows (each of them with a number from 0 to 999):

In [ ]:
DFrange = spark.range(1000).toDF("number")

After this, we can propose to make any kind of *transformation*. For example, we might want to retrieve only even values:

In [ ]:
DFdivisionBy2 = DFrange.where("number % 2 = 0")

But, so far, we really have done nothing! We have declared how our DFs would look like, but the computation has not taken place, yet. As execution is carried out lazily, it is not until we call an *action* that computation is performed. For example, by counting the number of rows in the resulting DF:

In [ ]:
DFdivisionBy2.count()

## A more complex example
In this second example, we will be using data like this:

In [ ]:
!head ~/data/summary_flights/2015-summary.csv

Let's declare the load of the data from the csv file. We instruct Spark to read the data in CSV format from a given path, and we ask it to infer both the schema of the data and the header (column names).

In [ ]:
flightData = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
.csv("data/summary_flights/*.csv") #to load all of it  .csv("data/summary_flights/2015-summary.csv") # 2015 data. Use 

We can have a look to the inferred information about the dataset:

In [ ]:
flightData.dtypes

When we use a Spark session, we are implicitly using DataFrames. We could force the declaration of RDDs by using the SparkContext.

Then, we could declare the DF from that RDD and we end up with the same result.

In [ ]:
rdd = spark.sparkContext.textFile('data/summary_flights/*.csv')
flightData_b = spark.read.option("inferSchema", "true").option("header", "true").csv(rdd)
flightData_b.dtypes

You might want to control the number of partitions in which your dataset is divided. How many are we using?

In [ ]:
flightData_b.rdd.getNumPartitions()

In [ ]:
flightData_b.show()

Let's change that. I want to use 4 partitions:

In [ ]:
flightData_b = flightData_b.repartition(4)
flightData_b.rdd.getNumPartitions()

In [ ]:
flightData_b.show()

Maybe you want to make a partition taking into account the values of a specific variable:

In [ ]:
from pyspark.sql.functions import col
flightData_b = flightData_b.repartition(5, col("ORIGIN_COUNTRY_NAME"))
print(flightData_b.rdd.getNumPartitions())
flightData_b.explain()

In [ ]:
flightData_b.show()

Partitioning your DataFrame might seem a low-level detail that you don't want to thing about. However, note that the number (and type of partitions) might have a clear impact on the performance (e.g., think of joins or similar operations).

## Using our DF
We can call an *action* to start getting some results and processing the data:

In [ ]:
flightData.sort("count","ORIGIN_COUNTRY_NAME").take(10)

This transformation can be inspected: what is doing Spark to compute it? Use `explain` to get the execution plan:

In [ ]:
flightData.sort("count","ORIGIN_COUNTRY_NAME").explain()

It's perfectly normal that you don't understand the major part of the previous output. At a first glance, it is a bit confusing. But, if you start paying attention, you'll realize that it forms a tree. Different operations of your query are found in different lines as organized by the plan, including implementation details. Other operations required to compute the query (e.g., partitioning) are included too. Do you remember the definition of query evaluation plan from the first block of the subject?

Note that, by default, it is using the Exchange operator with 200 partitions. As the dataset is not that large, to preserve communications, it might be a better idea to use less partitions. Let's modify the default value of no. partitions during shuffles:

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData.sort("count","ORIGIN_COUNTRY_NAME").explain()

Now, the plan is the same, but we will be creating only 5 partitions in the sort operatin (wide dependency).

## DataFrames and SQL

Did you know that we can use SQL to query our DataFrames?!

First, we need to register the DataFrame as a table or view, and then we can just query it.

In [ ]:
flightData.createOrReplaceTempView("tab_flight_data")
flightData.schema

Now, we can query the table `tab_flight_data` using regular SQL and `spark.sql()` function. Let's replicate the previous query:

In [ ]:
spark.sql("""
SELECT *
FROM tab_flight_data
SORT BY count, ORIGIN_COUNTRY_NAME
""").show(10)

For example, let's execute this query and take just the first row:

In [ ]:
spark.sql("SELECT max(count) as maxcount from tab_flight_data GROUP BY ORIGIN_COUNTRY_NAME").take(1)

Or we can use the SQL standard `LIMIT` clause for selecting the number of elements that we want (do not use `take()`):

In [ ]:
### YOUR CODE HERE ###


We can do the same with DataFrames and its API's functions as:

In [ ]:
flightData.groupBy("ORIGIN_COUNTRY_NAME")\
.max("count")\
.withColumnRenamed("max(count)", "maxcount")\
.select("maxcount")\
.take(1)

Let's try something more complex:

In [ ]:
sqlWay = spark.sql("""
SELECT ORIGIN_COUNTRY_NAME, avg(count) as origin_avg
FROM tab_flight_data
GROUP BY ORIGIN_COUNTRY_NAME
ORDER BY origin_avg DESC
LIMIT 5
""")
sqlWay.show()

In [ ]:
#### YOUR CODE HERE ####
from pyspark.sql.functions import desc
dfWay = flightData\
...

dfWay.show()

In fact, Spark will compile and optimize both DF and SQL queries in the same way. To the point that they might end up with exactly the same execution plan:

In [ ]:
sqlWay.explain()
dfWay.explain()

Note that both query types (DF functions and SQL) are **not incompatible**!! We can combine both and they will be efficiently compile into a single execution plan:

In [ ]:
dfPlusSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as agg_count
FROM tab_flight_data GROUP BY DEST_COUNTRY_NAME
""")\
  .where("DEST_COUNTRY_NAME like 'S%'")\
  .where("agg_count > 10")

dfPlusSql.show()
dfPlusSql.explain()

Pay attention to the physical plan. It is doing aggregation twice. Where do you see it? Why is it done twice? With which objective?

. ### YOUR ANSWER HERE ###

## Joins
Let's see how we can make joins of different DataFrames in Spark.

To do so, let's create first of all a few toy DataFrames:

In [ ]:
person = spark.createDataFrame([
    (0, "John Von Neumann", [0,2], 10),
    (1, "Judea Pearl", [1,2,3], 10),
    (3, "Alan Turing", [0,2], 10),
    (2, "Frances Allen", [0,1,2,3], 10),
    (4, "Joan Camps", [0], 20),
    (5, "Bruna Oliva", [2], 20),
    (7, "Marc Soler", [1], 20),
    (6, "Oriol Serra", [2,3], 20),
    (8, "Laia Marques", [0], 20)])\
  .toDF("pid", "name", "program", "status")
program = spark.createDataFrame([
    (0, "Bachelors", "Computer Science", "UB"),
    (2, "Masters", "Data Science", "UB"),
    (1, "Masters", "Artificial Intelligence", "UB/UPC/URV"),
    (3, "Ph.D.", "Mathematics and Computer Science", "UB")])\
  .toDF("did", "degree", "title", "school")
status = spark.createDataFrame([
    (20, "Student"),
    (11, "Dean"),
    (10, "Teacher")])\
  .toDF("sid", "status")

We register them as relational tables to be able to use SQL:

In [ ]:
person.createOrReplaceTempView("tab_person")
program.createOrReplaceTempView("tab_program")
status.createOrReplaceTempView("tab_status")

If we want to make a join using only DF functions, we need to use an expression that relates columns of the DFs that we want to join:

In [ ]:
joinExpression = person["status"] == status['sid']
#wrong: person["name"] == program["school"]

And just run the join as:

In [ ]:
person.join(status, joinExpression).show()

This is equivalent to the SQL query:

In [ ]:
### YOUR CODE HERE ###


You might be wondering how these are actually executed by Spark. Let's have a look to its execution plan:

In [ ]:
person.join(status, joinExpression).explain()

Apart from the standard join, we can use other types of joins. See, for example, this outer join:

In [ ]:
person.join(status, joinExpression, "outer").show()
spark.sql("SELECT * FROM tab_person FULL OUTER JOIN tab_status "+
          "ON tab_person.status = tab_status.sid").show()

Note that the relationship between people and programs is 1-to-n. That is, each person can be related to one or more programs (e.g., professors teaching in more than one degree). To make a join through that relationship, we need to pay special attention:

In [ ]:
from pyspark.sql.functions import expr
person.join(program, expr("array_contains(program, did)")).show()

How would you do this query with SQL?

In [ ]:
### YOUR CODE HERE ###


We are done here! Let's just close Spark's session:

In [ ]:
spark.stop()